In [3]:
# document_loaders를 통해 다양한 문서를 읽어올 수 있다.
from langchain.llms.ollama import Ollama
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = Ollama(
  model="llama3:latest",
  temperature=0.1
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=200, # 각 청크의 최대 길이(200개의 문자)
  chunk_overlap=50 # 청크간의 중첩 길이(이전 50개의 문자)
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OllamaEmbeddings(
  base_url="http://localhost:11434",  # 모델 서버의 URL
  model="llama3:latest"
)

cache_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)


vectorstore = Chroma.from_documents(docs, cache_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}"),
  ("human", "{question}")
])

# list of docs
# for doc in list of docs | prompt | llm
# for response in list of llms response | put them all together
# final doc | prompt | llm

map_doc_prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      """
      Use the following protion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
      ------
      {context}
      """
    ),
    ("human", "{question}")
  ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
  documents = inputs['documents']
  question = inputs['question']
  return "\n\n".join(
    map_doc_chain.invoke(
      {"context": doc.pㅁage_content,"question": question}
    )
    for doc in documents
    )

map_chain = { "documents": retriver, "question": RunnablePassthrough() } | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages([
  (
    "system",
    """
    Given the following extracted parts of a long docment and a question, create a final answer.
    If you don't know the answer, just say that you don't know. Don't try to make up an answer.
    ------
    {context}
    """,
  ),  
  ("human", "{question}")
])

chain = { "context": map_chain, "question": RunnablePassthrough() } | final_prompt | llm

 

chain.invoke("카카오페이는 무슨 서비스입니까?")


Created a chunk of size 211, which is longer than the specified 200
Created a chunk of size 212, which is longer than the specified 200
Created a chunk of size 217, which is longer than the specified 200
Created a chunk of size 222, which is longer than the specified 200


'Based on the provided text, I can answer the question:\n\nThe service is a payment method that allows users to send already charged KakaoPay money by scanning a QR code.'